In [5]:
import os
import pandas as pd
import plotly.graph_objs as go

In [6]:
# ram_sizes = [1, 2, 4, 8, 12, 16]
ram_sizes = [1]
mig_methods = ['Vanilla', 'XBZRLE', 'Compress', 'HBFDP_SHA1','HBFDP_MD5', 'HBFDP_Murmur3', 'XBZRLE_HBFDP_SHA1', 'XBZRLE_HBFDP_MD5', 'XBZRLE_HBFDP_Murmur3', 'Compress_HBFDP_SHA1', 'Compress_HBFDP_MD5', 'Compress_HBFDP_Murmur3']


In [7]:
def mig_time_plot(plot_column="Total-Time", y_title="Total Migration Time (s)", workload="Sysbench"):
    mig_data = {}
    for mig_method in mig_methods:
        mig_data[mig_method] = {"data": [], "deviation": []}
        for ram_size in ram_sizes:
            ram_size_mb = str(ram_size*1024)
            if os.path.exists(f'./data/{mig_method}/{workload}/{ram_size_mb}.csv'):
                data = pd.read_csv(f'./data/{mig_method}/{workload}/{ram_size_mb}.csv')
                filtered_data = data[data['Outlier'] != 'Yes']
                if plot_column == "Total-Time" or plot_column == "Downtime":
                    mig_data[mig_method]["data"].append(filtered_data[plot_column].mean()/1000)
                    mig_data[mig_method]["deviation"].append(filtered_data[plot_column].std()/1000)
                else:
                    mig_data[mig_method]["data"].append(filtered_data[plot_column].mean())
                    mig_data[mig_method]["deviation"].append(filtered_data[plot_column].std())
    
    data = pd.DataFrame({'RAM Size (GB)': [ str(ram_size) for ram_size in ram_sizes]})
    
    count=1
    for mig_method in mig_methods:
        if len(mig_data[mig_method]["data"]) == 1:
            data.insert(column=mig_method, value=mig_data[mig_method]["data"], loc=count)  
            count += 1
    
    # print(data)

    layout = go.Layout(
                    xaxis=dict(title='RAM Size (GB)'),
                    yaxis=dict(title=y_title),
                    barmode='group',
                    template='simple_white'
                    )
    
    trace = []
    
    for mig_method in mig_methods:
        name = ""
        for word in mig_method.split("_"):
            name = f'{name} {word}'
            
        if len(mig_data[mig_method]["data"]) == 1:
            trace.append(go.Bar(x=data['RAM Size (GB)'], y=data[mig_method], name=f'{name} Pre-Copy', error_y=dict(type='data', array=mig_data[mig_method]["deviation"])))

    fig = go.Figure(data=trace, layout=layout)

    fig.update_layout(
        width=777,  # 0.45 text width
        height=700,
        font=dict(family="Arial", size=17.2, color="black"),
        margin=dict(t=15, r=5, l=70, b=45, autoexpand=True),
        yaxis_title_standoff=10,
        xaxis_title_standoff=8,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            # Align center
            y=-0.5,
        ),
    )
    fig.write_image(f"./RAM-Size-vs-{plot_column}.{workload}.png",scale=4, width=777, height=700)

    fig.show()

In [8]:
mig_time_plot("Total-Time", "Total Migration Time (s)", "Memcached")